# Load the data


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('glass.csv')

In [3]:
data

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data = data.drop('Type', axis=1)

stan_data = scaler.fit_transform(data)

In [5]:
stan_data.shape

(214, 9)

# Calculating Mahalanobis distances for every point

In [6]:
list_ = np.zeros((stan_data.shape[0], stan_data.shape[0]))

In [7]:
mean = np.mean(stan_data, axis=0)
cov = np.cov(stan_data, rowvar=False)
inv_cov = np.linalg.inv(cov)

In [8]:
# list_ = np.zeros((stan_data.shape[0], stan_data.shape[0]))

for sample in range(len(stan_data)):
    for nbr in range(len(stan_data)):
        if sample != nbr:
            temp = np.dot((stan_data[sample]-stan_data[nbr]).T, inv_cov)
            temp = np.dot(temp,stan_data[sample]-stan_data[nbr])
            list_[sample][nbr] = np.sqrt(temp)
        else:
            list_[sample][nbr] = 0

Mahalanobis_score = np.diagonal(np.sqrt((stan_data-mean) @ inv_cov @ (stan_data-mean).T))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16312\2677110458.py:12: RuntimeWarning: invalid value encountered in sqrt
  Mahalanobis_score = np.diagonal(np.sqrt((stan_data-mean) @ inv_cov @ (stan_data-mean).T))


In [9]:
Mahalanobis_score

array([2.19313092, 1.59918789, 1.3835073 , 2.50289121, 1.34550441,
       2.44744587, 1.39684586, 1.58391194, 1.86838199, 1.77689004,
       2.30619262, 1.57657447, 2.28530121, 1.80391866, 1.81511641,
       1.54498811, 1.93673915, 2.8015078 , 1.57632799, 2.0919544 ,
       1.73889108, 3.5608793 , 1.43847933, 1.95029466, 1.12887921,
       1.50964054, 1.17664837, 1.55365853, 1.70126   , 1.04520646,
       1.9404724 , 1.52963537, 2.01149334, 2.34600753, 1.36447561,
       2.49208361, 1.86500342, 1.34847742, 2.77613458, 2.77613458,
       1.44718737, 1.50779559, 0.97269955, 2.32609031, 2.65933263,
       1.36685646, 1.5668403 , 4.91764383, 1.98433951, 1.50896846,
       2.59601667, 1.46379688, 1.24285467, 1.27416185, 0.79294364,
       2.63883619, 6.03389189, 2.48320918, 2.10923647, 1.3368424 ,
       1.19839099, 2.27614639, 1.70358065, 2.59761887, 1.78587657,
       1.58452261, 1.79569726, 2.30550982, 1.7578118 , 2.34729563,
       3.38149557, 3.25531967, 1.28659935, 1.19214355, 1.36956

# Calculating LOF score

In [10]:
k = 4
k_distance = []
k_Neigbour = []
# Step1 --> calculate K-Distance and K-Neighbour
for i in range(stan_data.shape[0]):
    lst = list_[i]
    sorted_lst = np.sort(lst)
    count = 0
    temp_lst = []
    for j in range(len(sorted_lst)):
        if sorted_lst[j] <= k:
            count += 1
            temp_lst.append(j)
        else:
            break
    k_distance.append(sorted_lst[count])
    k_Neigbour.append(temp_lst)
    
# Step2 --> Reachibility Distance 
reachibility_distance = np.zeros((stan_data.shape[0], stan_data.shape[0]))
for sample in range(len(stan_data)):
    for nbr in range(len(stan_data)):
        if sample != nbr:
            reachibility_distance[sample][nbr] = max(k_distance[nbr], list_[sample][nbr])
        else:
            reachibility_distance[sample][nbr] = 0
            
# Step3 --> Local Reachibility Distance
local_reachibility_distance = []
for i in range(stan_data.shape[0]):
    count = 0
    for j in k_Neigbour[i]:
        count += (reachibility_distance[i][j]/len(k_Neigbour[i]))
        
    local_reachibility_distance.append(count)
    
# Step4 --> Local Outlier Factor
lofs = []
for i in range(stan_data.shape[0]):
    count = 0
    for j in k_Neigbour[i]:
        count += local_reachibility_distance[j]
        
    lofs.append(count/(local_reachibility_distance[i] * len(k_Neigbour[i])))
    

In [11]:
np.mean(lofs)
np.std(lofs)

0.12346490824798838

In [12]:
lofs

[1.0434031164331345,
 1.040145185791828,
 1.043572318207965,
 1.0412409507807314,
 1.0450782604269395,
 1.0328708924601513,
 1.0448176645412506,
 1.0440906664535279,
 1.0398406704795304,
 1.0392053807769406,
 1.0343080683923087,
 1.043164236083381,
 1.0357714133988158,
 1.043203114600972,
 1.0382355448385223,
 1.046607859787843,
 1.0398981820812034,
 1.0314718895903254,
 1.0442362268690908,
 1.031649693214981,
 1.04859445632111,
 1.0061342176189858,
 1.0504193710229823,
 1.0351201216321761,
 1.049628316570179,
 1.0427565635121765,
 1.051067413830664,
 1.0499136543093688,
 1.0445850587963994,
 1.0510944592037743,
 1.0394622183533455,
 1.0448195876370405,
 1.0439972137186522,
 1.0258023716827283,
 1.0468222425239073,
 1.038808971131854,
 1.0409967113410108,
 1.046366415891902,
 1.029854580683387,
 1.029854580683387,
 1.048447898685509,
 1.0501121226424843,
 1.0503021406273019,
 1.0440587634053489,
 1.0380956906474634,
 1.0515118912173445,
 1.0497436481100317,
 0.9204976880399427,
 1.0506

In [13]:
from skimage.filters import threshold_otsu
lofs = np.array(lofs)
threshold = threshold_otsu(lofs)
outlier_mask = lofs > threshold

print("Number of outliers:", np.sum(outlier_mask))
print("Number of inliers:", np.sum(~outlier_mask))

Number of outliers: 198
Number of inliers: 16


# Otsu Threshold

In [14]:
sorted_lofs_id = lofs.argsort()

min_row = 1e+10
final_value = 0
i = lofs[sorted_lofs_id[0]]
while i < lofs[sorted_lofs_id[-1]]:
        Right_Class = lofs[sorted_lofs_id >= i]
        Left_Class = lofs[sorted_lofs_id < i]
    
        w_1 = len(Right_Class)/len(sorted_lofs_id)
        w_0 = len(Left_Class)/len(sorted_lofs_id)
        var_left = (np.std(Left_Class))**2
        var_right = (np.std(Right_Class))**2

        row = w_0*var_left + w_1*var_right
        if min_row > row:
            min_row = row
            final_value = i
        
        i += (lofs[sorted_lofs_id[-1]] - lofs[sorted_lofs_id[0]])/10

final_value

1.0565898870196617

Threshold is 1.0565898870196617

In [15]:
count = 0
for i in lofs:
    if i <= final_value:
        count += 1
len(lofs) - count

1

In [34]:
sorted_lofs_id = Mahalanobis_score.argsort()

min_row = 1e+10
final_value = 0
i = Mahalanobis_score[sorted_lofs_id[0]]
while i < Mahalanobis_score[sorted_lofs_id[-1]]:
        Right_Class = Mahalanobis_score[sorted_lofs_id >= i]
        Left_Class = Mahalanobis_score[sorted_lofs_id < i]
    
        w_1 = len(Right_Class)/len(sorted_lofs_id)
        w_0 = len(Left_Class)/len(sorted_lofs_id)
        var_left = (np.std(Left_Class))**2
        var_right = (np.std(Right_Class))**2

        row = w_0*var_left + w_1*var_right
        if min_row > row:
            min_row = row
            final_value = i
        
        i += (Mahalanobis_score[sorted_lofs_id[-1]] - Mahalanobis_score[sorted_lofs_id[0]])/10
        
final_value -= 0.25

In [35]:
count = 0
for i in Mahalanobis_score:
    if i <= final_value:
        count += 1
count

32